<a href="https://colab.research.google.com/github/pholoubek/SpiderMat-O-Mat/blob/master/Spider_mat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [2]:
!pip install tensorboardcolab

  Running setup.py bdist_wheel for tensorboardcolab ... - done
  Stored in directory: /root/.cache/pip/wheels/33/cf/fe/750a8f7de59b74b0f40497c9c778ef06a968f319f0a3e85361
Successfully built tensorboardcolab


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import os
os.chdir('/content/drive/My Drive/AI/')

In [9]:
from tensorboardcolab import *
tbc=TensorBoardColab()

Wait for 8 seconds...
TensorBoard link:
http://fb136361.ngrok.io


In [0]:
import cv2
import sys
import numpy as np
import tensorflow as tf

# sess = tf.Session()
# sess.run(tf.global_variables_initializer())
tf.global_variables_initializer()

def parser(record):
    keys_to_features = {
        "image_raw": tf.FixedLenFeature([], tf.string),
        "label":     tf.FixedLenFeature([], tf.int64)
    }
    parsed = tf.parse_single_example(record, keys_to_features)
    image = tf.decode_raw(parsed["image_raw"], tf.uint8)
    image = tf.cast(image, tf.float32)
    label = tf.cast(parsed["label"], tf.int32)

    return {'image': image}, label

def input_fn(filenames):
  dataset = tf.data.TFRecordDataset(filenames=filenames, num_parallel_reads=40)
  dataset = dataset.apply(
      tf.data.experimental.shuffle_and_repeat(300, 1)
  )
  dataset = dataset.apply(
      tf.data.experimental.map_and_batch(parser, 100)
  )
  dataset = dataset.prefetch(buffer_size=2)
  return dataset


def train_input_fn():
    return input_fn(filenames=["train.tfrecords", "test.tfrecords"])

def val_input_fn():
    return input_fn(filenames=["val.tfrecords"])

def model_fn(features, labels, mode, params):
    num_classes = 2
    net = features["image"]

    net = tf.identity(net, name="input_tensor")
    
    net = tf.reshape(net, [-1, 224, 224, 1])    

    net = tf.identity(net, name="input_tensor_after")

    net = tf.layers.conv2d(inputs=net, name='layer_conv1',
                           filters=64, kernel_size=2,
                           padding='SAME', activation=tf.nn.relu)  
    net = tf.layers.max_pooling2d(inputs=net, pool_size=2, strides=2)

    net = tf.layers.conv2d(inputs=net, name='layer_conv2',
                           filters=64, kernel_size=2,
                           padding='SAME', activation=tf.nn.relu)
    net = tf.layers.max_pooling2d(inputs=net, pool_size=2, strides=2)

    net = tf.layers.conv2d(inputs=net, name='layer_conv3',
                           filters=64, kernel_size=2,
                           padding='same', activation=tf.nn.relu)
    net = tf.layers.max_pooling2d(inputs=net, pool_size=2, strides=2)
    
    net = tf.layers.conv2d(inputs=net, name='layer_conv4',
                       filters=128, kernel_size=2,
                       padding='same', activation=tf.nn.relu)
    net = tf.layers.max_pooling2d(inputs=net, pool_size=2, strides=2)
    
#     net = tf.layers.conv2d(inputs=net, name='layer_conv5',
#                        filters=128, kernel_size=,
#                        padding='same', activation=tf.nn.relu)  
#     net = tf.layers.max_pooling2d(inputs=net, pool_size=2, strides=2)    

    net = tf.contrib.layers.flatten(net)
    
#     net = tf.layers.dense(inputs=net, name='layer_fc1',
#                         units=1024, activation=tf.nn.relu)
    
    net = tf.layers.dense(inputs=net, name='layer_fc2',
                    units=128, activation=tf.nn.relu)
    
#     net = tf.layers.dropout(net, rate=0.4, noise_shape=None, 
#                         seed=None, training=(mode == tf.estimator.ModeKeys.TRAIN))
    
    net = tf.layers.dense(inputs=net, name='layer_fc3',
                    units=128, activation=tf.nn.relu)
    
    net = tf.layers.dense(inputs=net, name='layer_fc_4',
                        units=num_classes)

    logits = net
    y_pred = tf.nn.softmax(logits=logits)

    y_pred = tf.identity(y_pred, name="output_pred")

    y_pred_cls = tf.argmax(y_pred, axis=1)

    y_pred_cls = tf.identity(y_pred_cls, name="output_cls")

    if mode == tf.estimator.ModeKeys.PREDICT:
        spec = tf.estimator.EstimatorSpec(mode=mode,
                                          predictions=y_pred_cls)
    else:
        cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=labels,
                                                                       logits=logits)
        loss = tf.reduce_mean(cross_entropy)

        optimizer = tf.train.AdamOptimizer(learning_rate=params["learning_rate"])
        
        train_op = optimizer.minimize(
            loss=loss, global_step=tf.train.get_global_step())

        metrics = {
            "accuracy": tf.metrics.accuracy(labels, y_pred_cls)
        }

        spec = tf.estimator.EstimatorSpec(
            mode=mode,
            loss=loss,
            train_op=train_op,
            eval_metric_ops=metrics,
        )
        
    return spec

  


In [0]:
model = tf.estimator.Estimator(model_fn=model_fn,
                               params={"learning_rate": 0.000009},
                               model_dir="/content/drive/My Drive/AI/spider_model25/")

epoch = 0
avg = 0
while (epoch < 1000):
    model.train(input_fn=train_input_fn, steps=100)
    result = model.evaluate(input_fn=val_input_fn)
    
    tbc.save_value("Accuracy/Epoch", "gf1", epoch, result["accuracy"]*100)
    tbc.save_value("Loss/Epoch", "gf2", epoch, result["loss"])
    
    
    tbc.flush_line("gf1")
    tbc.flush_line("gf2")
    tbc.close()

    sys.stdout.flush()
    epoch = epoch + 1

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/content/drive/My Drive/AI/spider_model25/', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7ff4b6782a58>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook

In [6]:
from random import shuffle
import glob
import sys
import cv2
import numpy as np
import tensorflow as tf

def _int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))
    
def _bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def load_image(addr):
    img = cv2.imread(addr)
    if img is None:
        return None
    img = cv2.resize(img, (224, 224), interpolation=cv2.INTER_CUBIC)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    return img
 
def createDataRecord(out_filename, addrs, labels):
    # open the TFRecords file
    writer = tf.python_io.TFRecordWriter(out_filename)
    for i in range(len(addrs)):
        # print how many images are saved every 1000 images
        if not i % 50:
            print('Train data: {}/{}'.format(i, len(addrs)))
            sys.stdout.flush()

        # Load the image
        img = load_image(addrs[i])
        label = labels[i]

        #  just sentinel to prevent crash
        if img is None:
            continue

        # Create a feature
        feature = {
            'image_raw': _bytes_feature(img.tostring()),
            'label': _int64_feature(label)
        }
        # Create an example protocol buffer
        example = tf.train.Example(features=tf.train.Features(feature=feature))
        
        # Serialize to string and write on the file
        writer.write(example.SerializeToString())
        
    writer.close()
    sys.stdout.flush()

dataset_path = '/content/drive/My Drive/AI/dataset/*/*'
# dataset_path = '/home/peterson/Documents/School/Fall 2018/ML/Project/dataset/*/*'
# read addresses and labels from the 'train' folder
addrs = glob.glob(dataset_path)

labels = [1 if 'spiders' in addr else 0 for addr in addrs]

# to shuffle data
c = list(zip(addrs, labels))
shuffle(c)
addrs, labels = zip(*c)
    
# Divide the data into 60% train, 20% validation, and 20% test
train_addrs = addrs[0:int(0.65*len(addrs))]
train_labels = labels[0:int(0.65*len(labels))]
val_addrs = addrs[int(0.65*len(addrs)):int(0.8*len(addrs))]
val_labels = labels[int(0.65*len(addrs)):int(0.8*len(addrs))]
test_addrs = addrs[int(0.8*len(addrs)):]
test_labels = labels[int(0.8*len(labels)):]

createDataRecord('train.tfrecords', train_addrs, train_labels)
createDataRecord('val.tfrecords', val_addrs, val_labels)
createDataRecord('test.tfrecords', test_addrs, test_labels)

Train data: 0/1147
Train data: 50/1147
Train data: 100/1147
Train data: 150/1147
Train data: 200/1147
Train data: 250/1147
Train data: 300/1147
Train data: 350/1147
Train data: 400/1147
Train data: 450/1147
Train data: 500/1147
Train data: 550/1147
Train data: 600/1147
Train data: 650/1147
Train data: 700/1147
Train data: 750/1147
Train data: 800/1147
Train data: 850/1147
Train data: 900/1147
Train data: 950/1147
Train data: 1000/1147
Train data: 1050/1147
Train data: 1100/1147
Train data: 0/265
Train data: 50/265
Train data: 100/265
Train data: 150/265
Train data: 200/265
Train data: 250/265
Train data: 0/354
Train data: 50/354
Train data: 100/354
Train data: 150/354
Train data: 200/354
Train data: 250/354
Train data: 300/354
Train data: 350/354
